In [1]:
from finalmodel import EncoderDecoder
from utils import euc_dist_keras
import pickle
import numpy as np
import open3d as o3d

In [2]:
model = EncoderDecoder()
model.compile(optimizer='adam', loss=euc_dist_keras, metrics = ['accuracy'])

In [ ]:
def inferr(model, weight, inference_file):
    
    mesh = o3d.io.read_triangle_mesh(inference_file)
    pcd = mesh.sample_points_uniformly(number_of_points = 1000)
    pcd_array = np.asarray(pcd.points)
    model.build(input_shape = (None, 1000,3))
    model.load_weights(weight)
    pcd_input = np.expand_dims(pcd_array, axis=0)
    predicted_array = model.predict(pcd_input)
    output = predicted_array[0]
    print(output.shape)
    pcd_output = o3d.geometry.PointCloud()
    pcd_output.points = o3d.utility.Vector3dVector(output)
    o3d.io.write_point_cloud("output.ply", pcd_output)
  
    tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd_output)
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd_output, 0.038, tetra_mesh, pt_map)
    o3d.io.write_triangle_mesh("outputfile.obj", rec_mesh)
    
    return pcd_output, rec_mesh

In [ ]:
inference_file = 'incompletePart.obj' # incomplete .obj file to complete
weight = 'modelSaved' # saved file of the trained model
model = model

In [ ]:
output_pointcloud, output_mesh = inferr(model, weight, inference_file)

In [ ]:
def visualize_object(mesh_obj):
    o3d.visualization.draw_geometries([any_object])

In [ ]:
# visualize mesh
visualize_object(output_mesh)

In [ ]:
# visualize point cloud
visualize_object(output_pointcloud)